In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime
from statistics import mean

In [3]:
def smoothing(data):
    series = data
    # Tail-rolling average transform
    rolling = series.rolling(window=8)
    rolling_mean = rolling.mean()
    return rolling_mean

In [4]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [5]:
def mageCalculation(df, std=1):
    
    #extracting glucose values and incdices
    glucs = df['GlucoseValue'].to_list()
    indices = [1*i for i in range(len(glucs))]
    stdev = std
    
    # detection of local minima and maxima
    x = indices
    gvs = glucs
    # local min & max
    a = np.diff(np.sign(np.diff(gvs))).nonzero()[0] + 1      
    # local min
    valleys = (np.diff(np.sign(np.diff(gvs))) > 0).nonzero()[0] + 1 
    # local max
    peaks = (np.diff(np.sign(np.diff(gvs))) < 0).nonzero()[0] + 1         
    # +1 due to the fact that diff reduces the original index number

    #storing the local minima and maxima to identify and remove turning points
    excursion_points = pd.DataFrame(columns=['Index', 'Timestamp', 'GlucoseValue', 'Type'])
    k=0
    for i in range(len(peaks)):
        excursion_points.loc[k] = [peaks[i]] + [df['Display Time'][k]] + [df['GlucoseValue'][k]] + ["P"]
        k+=1

    for i in range(len(valleys)):
        excursion_points.loc[k] = [valleys[i]] + [df['Display Time'][k]] + [df['GlucoseValue'][k]] + ["V"]
        k+=1

    excursion_points = excursion_points.sort_values(by=['Index'])
    excursion_points = excursion_points.reset_index(drop=True)
    # display(excursion_points)


    # selecting turning points
    turning_points = pd.DataFrame(columns=['Index', 'Timestamp', 'GlucoseValue', 'Type'])
    k=0
    for i in range(stdev,len(excursion_points.Index)-stdev):
        positions = [i-stdev,i,i+stdev]
        for j in range(0,len(positions)-1):
            if(excursion_points.Type[positions[j]] == excursion_points.Type[positions[j+1]]):
                if(excursion_points.Type[positions[j]]=='P'):
                    if excursion_points.GlucoseValue[positions[j]]>=excursion_points.GlucoseValue[positions[j+1]]:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1
                    else:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1
                else:
                    if excursion_points.GlucoseValue[positions[j]]<=excursion_points.GlucoseValue[positions[j+1]]:
                        turning_points.loc[k] = excursion_points.loc[positions[j]]
                        k+=1
                    else:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1

    if len(turning_points.index)<10:
        turning_points = excursion_points.copy()
        excursion_count = len(excursion_points.index)
    else:
        excursion_count = len(excursion_points.index)/2



    turning_points = turning_points.drop_duplicates(subset= "Index", keep= "first")
    turning_points=turning_points.reset_index(drop=True)
    excursion_points = excursion_points[excursion_points.Index.isin(turning_points.Index) == False]
    excursion_points = excursion_points.reset_index(drop=True)
        # display(turning_points)

    # calculating the MAGE score
    mage = turning_points.GlucoseValue.sum()/excursion_count
    

    return round(mage,3)

In [28]:
def saveExcusrions(subjectId, df, std=1):    
    #extracting glucose values and incdices
    glucs = df['GlucoseValue'].to_list()
    indices = [1*i for i in range(len(glucs))]
    stdev = std
    
    # detection of local minima and maxima
    x = indices
    gvs = glucs
    # local min & max
    a = np.diff(np.sign(np.diff(gvs))).nonzero()[0] + 1      
    # local min
    valleys = (np.diff(np.sign(np.diff(gvs))) > 0).nonzero()[0] + 1 
    # local max
    peaks = (np.diff(np.sign(np.diff(gvs))) < 0).nonzero()[0] + 1         
    # +1 due to the fact that diff reduces the original index number

    #storing the local minima and maxima to identify and remove turning points
    excursion_points = pd.DataFrame(columns=['Index', 'Timestamp', 'GlucoseValue', 'Type'])
    k=0
    for i in range(len(peaks)):
        excursion_points.loc[k] = [peaks[i]] + [df['Display Time'][k]] + [df['GlucoseValue'][k]] + ["P"]
        k+=1

    for i in range(len(valleys)):
        excursion_points.loc[k] = [valleys[i]] + [df['Display Time'][k]] + [df['GlucoseValue'][k]] + ["V"]
        k+=1

    excursion_points = excursion_points.sort_values(by=['Index'])
    excursion_points = excursion_points.reset_index(drop=True)
    # display(excursion_points)


    # selecting turning points
    turning_points = pd.DataFrame(columns=['Index', 'Timestamp', 'GlucoseValue', 'Type'])
    k=0
    for i in range(stdev,len(excursion_points.Index)-stdev):
        positions = [i-stdev,i,i+stdev]
        for j in range(0,len(positions)-1):
            if(excursion_points.Type[positions[j]] == excursion_points.Type[positions[j+1]]):
                if(excursion_points.Type[positions[j]]=='P'):
                    if excursion_points.GlucoseValue[positions[j]]>=excursion_points.GlucoseValue[positions[j+1]]:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1
                    else:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1
                else:
                    if excursion_points.GlucoseValue[positions[j]]<=excursion_points.GlucoseValue[positions[j+1]]:
                        turning_points.loc[k] = excursion_points.loc[positions[j]]
                        k+=1
                    else:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1

    if len(turning_points.index)<10:
        turning_points = excursion_points.copy()
        excursion_count = len(excursion_points.index)
    else:
        excursion_count = len(excursion_points.index)/2



    turning_points = turning_points.drop_duplicates(subset= "Index", keep= "first")
    turning_points=turning_points.reset_index(drop=True)
    excursion_points = excursion_points[excursion_points.Index.isin(turning_points.Index) == False]
    excursion_points = excursion_points.reset_index(drop=True)
    
    fig = plt.figure(figsize=(36, 18))
    plt.plot(x, gvs, color='grey')

    plt.xlabel('Indexes')
    plt.ylabel('Glucose Values')

    a = []
    b = []
    for i in excursion_points.Index:
        a.append(x[i])
        b.append(gvs[i])
    #     plt.plot(x[i], gvs[i], "o", label="min", color='orange')

    plt.plot(a,b,color="red")

    plt.title('Glucose Value Excursions for '+str(subjectId)+' with std dev '+str(stdev))
    fig.savefig('/Users/snehgajiwala/Desktop/NCSA_genomics/Python - notebooks/TSForecasting/Data/Excursions/'+str(subjectId)+'_'+str(stdev), dpi=fig.dpi)
    plt.close()
    

In [21]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/TSForecasting/Data/consolidatedDataForPackage.csv")
data['Display Time'] = data['Display Time'].apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

for subjectId, df in tqdm(data.groupby('subjectId')):
    
    df = fullDay(df)
    length = df['Display Time'].iloc[-1]-df['Display Time'].iloc[0]
    length = length.round("d")
    days = length.days

    df['GlucoseValue'] = smoothing(df['GlucoseValue'])
    df = df[df['GlucoseValue'].notna()]
    df = df.reset_index(drop=True)

    dates = []
    for i in range(len(df.index)):
        dates.append(df['Display Time'][i].date())
    df['Date'] = dates

    mage_daily = []
    for Date, xx in df.groupby('Date'):
        xx = xx.reset_index(drop=True)
        mage_daily.append(mageCalculation(xx,1))
    print(subjectId, round(mean(mage_daily)))



1636-69-001 31.953
1636-69-001-2 68.40733333333333
1636-69-026 83.9995
1636-69-028 75.3926
1636-69-032 27.1678
1636-69-035 123.1945
1636-69-048 44.8942
1636-69-053 53.23733333333333
1636-69-060 88.22033333333334
1636-69-064 94.3556
1636-69-069 66.85466666666667
1636-69-090 63.0484
1636-69-091 42.903333333333336
1636-69-100 60.3872
1636-69-104 58.6522
1636-69-104-2 74.1962
1636-69-107 38.777833333333334
1636-69-111 52.3622
1636-69-114 68.4486
1636-69-123 44.363142857142854
1636-70-1002 48.315
1636-70-1003 80.36716666666666
1636-70-1005 65.47685714285714
1636-70-1008 37.9016
1636-70-1010 65.91175
2133-001 55.12116666666667
2133-002 32.6415
2133-003 63.208666666666666
2133-004 90.95139999999999
2133-006 35.699600000000004
2133-007 90.06785714285714
2133-008 35.9872
2133-009 69.3228
2133-010 38.8215
2133-011 54.354
2133-012 57.95133333333333
2133-013 44.568714285714286
2133-015 61.125166666666665
2133-017 35.3906
2133-018 58.5546
2133-019 60.739666666666665
2133-019-2 28.17
2133-020 60.477

In [29]:
for subjectId, df in tqdm(data.groupby('subjectId')):
    
    df = fullDay(df)
    length = df['Display Time'].iloc[-1]-df['Display Time'].iloc[0]
    length = length.round("d")
    days = length.days

    df['GlucoseValue'] = smoothing(df['GlucoseValue'])
    df = df[df['GlucoseValue'].notna()]
    df = df.reset_index(drop=True)
    saveExcusrions(subjectId, df, std=1)
    